In [1]:

import importlib
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

import sccoda.datasets as scd

2024-12-19 21:32:07.845302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('metadata.csv')
df['samples'] = df['day']+'_'+df['condition'] 

In [3]:
pivot_df = df.pivot_table(index='samples', columns='celltypes', aggfunc='size', fill_value=0)
pivot_df['days'] = pivot_df.index.map(lambda x: x.split('_')[0])
pivot_df['condition'] = pivot_df.index.map(lambda x: x.split('_')[1]) 

In [4]:
row1, row2 = 0, 2  # Specify the indices of rows to exchange

# Swap rows and their indices
row1_data = pivot_df.iloc[row1].copy()
row2_data = pivot_df.iloc[row2].copy()

pivot_df.iloc[row1] = row2_data
pivot_df.iloc[row2] = row1_data

pivot_df.index.values[row1], pivot_df.index.values[row2] = pivot_df.index.values[row2], pivot_df.index.values[row1]


In [5]:
pivot_df['condition'] = pivot_df['condition'].map(lambda x: 'A_ctr' if x == 'ctr' else x)

In [5]:
data_all = dat.from_pandas(pivot_df, covariate_columns=['days',"condition"])
print(data_all) 

AnnData object with n_obs × n_vars = 9 × 15
    obs: 'days', 'condition'


In [15]:
data_all[data_all.obs['days'].isin(['D35','D56'])].X

ArrayView([[    0,     0,     0,   164,   308,  1088,   430,  2462,
                0,  3341,     0,  2166,     0,     0,   404],
           [    0,     0,     0,   142,   179,   432,  2733,  1241,
                0,  2018,     0,  1565,     0,   691,   785],
           [    0,     0,     0,   176,   844,  2972,  5275,  6727,
                0,  8250,     0,  3569,     0,     0,  1734],
           [    0,  1019,  3662,     0,   196,   770,   308,     0,
                0,  1693,     0,   639,   423,     0,  1400],
           [    0,   307,  3727,     0,   128,   924,  2312,     0,
                0,  1819,     0,   570,   513,   116,   696],
           [    0,  1924, 11715,     0,   191,  1420,  6316,     0,
                0,  4354,     0,  1091,  1130,     0,  2301]])

In [53]:
import re
delimiters = r'[/ ]'
re.split(delimiters, 'a/b c')

['a', 'b', 'c']

In [ ]:
for ct in data_all.var_names:
    viz.boxplots(data_all, feature_name="days", cell_types=[ct], args_boxplot={"linewidth":0.4, "widths":0.2}),# figsize=(15, 7))
    file_name = '_'.join(re.split(delimiters, ct))
    plt.savefig(f'./celltype_proportion_days/{file_name}.pdf')
    #plt.show()
    
viz.boxplots(data_all, feature_name="days",figsize=(15, 6), args_boxplot={"linewidth":0.4, "widths":0.2})
plt.savefig(f'./celltype_proportion_days/all_celltypes.pdf')

In [ ]:
for ct in data_all.var_names:
    viz.boxplots(data_all, feature_name="condition", cell_types=[ct], level_order=['FGF', 'ctr', 'BMP'], args_boxplot={"linewidth":0.4, "widths":0.2}),# figsize=(15, 7))
    file_name = '_'.join(re.split(delimiters, ct))
    plt.savefig(f'./celltype_proportion_conditions/{file_name}.pdf')
    #plt.show()

viz.boxplots(data_all, feature_name="condition",figsize=(15, 6), level_order=['FGF', 'ctr', 'BMP'], args_boxplot={"linewidth":0.4, "widths":0.2})
plt.savefig(f'./celltype_proportion_conditions/all_celltypes.pdf')

In [7]:
day_pairs = [['D35','D56'], ['D56', 'D97'], ['D35', 'D97']]
condition_pairs = [['ctr','FGF'], ['ctr','BMP'], ['BMP','FGF']]

sim_results = {}
sim_results['days'] = {}
sim_results['conditions'] = {}

for day_pair in day_pairs:
    model_day = mod.CompositionalAnalysis(data_all[data_all.obs['days'].isin(day_pair),:], formula="days")
    print(1)
    # Run MCMC
    sim_result = model_day.sample_hmc()
    print(2)
    sim_results['days']['_'.join(day_pair)] = sim_result

for cond_pair in condition_pairs:
    model_cond = mod.CompositionalAnalysis(data_all[data_all.obs['condition'].isin(cond_pair),:], formula="condition")
    print(1)
    # Run MCMC
    sim_result = model_cond.sample_hmc()
    print(2)
    sim_results['conditions']['_'.join(cond_pair)] = sim_result


Automatic reference selection! Reference cell type set to DMT
Zero counts encountered in data! Added a pseudocount of 0.5.
1


I0000 00:00:1734680175.721054   28719 service.cc:145] XLA service 0x7fd4c34235a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734680175.721111   28719 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-12-19 21:36:15.721404: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|          | 0/20000 [00:00<?, ?it/s]2024-12-19 21:36:15.865054: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734680176.810616   28719 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
100%|██████████| 20000/20000 [01

MCMC sampling finished. (105.283 sec)
Acceptance rate: 73.0%
2
Automatic reference selection! Reference cell type set to cycling cells
Zero counts encountered in data! Added a pseudocount of 0.5.
1


100%|██████████| 20000/20000 [01:20<00:00, 247.21it/s]


MCMC sampling finished. (103.642 sec)
Acceptance rate: 74.1%
2
Automatic reference selection! Reference cell type set to IP
Zero counts encountered in data! Added a pseudocount of 0.5.
1


...

MCMC sampling finished. (104.098 sec)
Acceptance rate: 60.3%
2
Automatic reference selection! Reference cell type set to IP
Zero counts encountered in data! Added a pseudocount of 0.5.
1


100%|██████████| 20000/20000 [01:30<00:00, 220.94it/s]


MCMC sampling finished. (117.524 sec)
Acceptance rate: 49.6%
2
Automatic reference selection! Reference cell type set to IP
Zero counts encountered in data! Added a pseudocount of 0.5.
1


100%|██████████| 20000/20000 [01:25<00:00, 233.73it/s]


MCMC sampling finished. (110.084 sec)
Acceptance rate: 57.2%
2


In [13]:
import os
for fdr in [2, 4, 6, 8]:
    for key in sim_results:
        for pair in sim_results[key]:
            sim_result = sim_results[key][pair]
            sim_result.set_fdr(est_fdr=0.1 * fdr)
            file_dir = f'/Users/albert/Desktop/Projects/Prompt_Optimization/sccoda_results_0_{fdr}/{pair}'
            os.makedirs(file_dir)
            sim_result.credible_effects().to_csv(file_dir + '/credible_effects.csv', index=True)
            sim_result.intercept_df.to_csv(file_dir + '/sccoda_intercept.csv', index=True)
            sim_result.effect_df.to_csv(file_dir +'/sccoda_effect.csv', index=True)

In [71]:
sim_results.credible_effects()

Covariate    Cell Type                         
days[T.D56]  CPEC                                  False
             CR                                     True
             DLN                                    True
             DMD                                    True
             DMT                                   False
             IP                                    False
             L/CGE like interneuron/progenitors    False
             PP/CR                                  True
             ULN                                   False
             aRG                                   False
             aRG/oRG                               False
             cycling cells                         False
             newborn neurons                        True
             oRG-like cells                        False
             stressed cells                        False
Name: Final Parameter, dtype: bool

In [76]:
sim_results.set_fdr(est_fdr=0.4)


In [77]:
sim_results.intercept_df.to_csv('sccoda_intercept.csv', index=True)

In [78]:
sim_results.effect_df.to_csv('sccoda_effect.csv', index=True)

In [79]:
sim_results.effect_df

Final Parameter  HDI 3%  \
Covariate        Cell Type                                                     
days[T.D56]      CPEC                                      -0.122212  -1.399   
                 CR                                         3.701424   2.657   
                 DLN                                        5.454001   4.394   
                 DMD                                       -1.754336  -3.030   
                 DMT                                       -0.123991  -0.922   
                 IP                                         0.000000   0.000   
                 L/CGE like interneuron/progenitors         0.391101  -0.050   
                 PP/CR                                     -4.441215  -5.510   
                 ULN                                        0.103620  -1.254   
                 aRG                                        0.000000  -0.456   
                 aRG/oRG                                   -0.067090  -1.468   
                 cycling cells                             -0.561206  -1.167   
                 newborn neurons                            3.467989   2.048   
                 oRG-like cells                            -0.063104  -1.691   
                 stressed cells                             0.931440   0.198   
days[T.D97]      CPEC                                       0.523623  -0.556   
                 CR                                         2.956247   1.758   
                 DLN                                        5.379873   4.355   
                 DMD                                       -1.576016  -2.995   
                 DMT                                       -2.463865  -3.551   
                 IP                                         0.000000   0.000   
                 L/CGE like interneuron/progenitors         0.000000  -0.647   
                 PP/CR                                     -4.455656  -5.525   
                 ULN                                        5.745698   4.813   
                 aRG                                       -4.810607  -5.956   
                 aRG/oRG                                    4.298858   3.330   
                 cycling cells                             -0.361663  -1.017   
                 newborn neurons                            3.122455   1.757   
                 oRG-like cells                            -0.846342  -2.720   
                 stressed cells                             0.000000  -0.911   
condition[T.BMP] CPEC                                       0.329393  -0.524   
                 CR                                         0.300902  -0.335   
                 DLN                                        0.000000  -0.147   
                 DMD                                        0.198014  -0.744   
                 DMT                                        0.095172  -0.783   
                 IP                                         0.000000   0.000   
                 L/CGE like interneuron/progenitors        -1.758598  -2.329   
                 PP/CR                                     -0.041704  -0.591   
                 ULN                                       -0.121285  -0.504   
                 aRG                                       -0.002013  -0.340   
                 aRG/oRG                                    0.044469  -0.434   
                 cycling cells                              0.000000  -0.372   
                 newborn neurons                            0.000000  -0.627   
                 oRG-like cells                            -0.321984  -1.988   
                 stressed cells                             0.000000  -0.746   
condition[T.FGF] CPEC                                       0.070005  -0.380   
                 CR                                        -0.375627  -1.012   
                 DLN                                       -0.013940  -0.285   
                 DMD                                 

In "credible_effect.csv", the final parameter column means whether the covariate has a credible effect of change on celltype proportion for each celltype. For example in D35_D56, D35 is the baseline and D56 is a covariate.

In "sccoda_effect.csv", this is a quantitative version of effect. The "final parameter" column corresponds to the true/false in the credible effects, where 0 means false and positive means true. The log-2 fold change column shows a similar pattern.

In "sccoda_intercept.csv", the "final parameter" column shows how the cell types are distributed without any active covariates. This file is not needed in comparing between different covariates.

You can check out https://sccoda.readthedocs.io/en/latest/getting_started.html for more interpretations.